In [46]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import random
import albumentations as A
import paste
from instaboostfast import InstaBoost
import json
import getcoco

In [47]:
data_dir = '../../data'
image_dir = 'images'
label_dir = 'labels'
crop_image_dir = 'c_images'
crop_label_dir = 'c_labels'
annotation_dir = 'annotations'
source_image_dir = 'src_images'
source_label_dir = 'src_labels'
aug_image_dir = 'new_images'
aug_label_dir = 'new_labels'
mask_dir = 'new_masks'

In [48]:
input_path = '../../data/src_images/0_aug/KS-FR-BLOIS_24330_1513711368928_1:1.png'
output_path = '../../data/new_images/KS-FR-BLOIS_24330_1513711368928_1:1.png'
input_image = cv2.imread(input_path)
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
plt.imshow(input_image)
plt.axis('off')
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [49]:
image = cv2.imread(input_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w = image.shape[:2]
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [50]:
# Set the lower and upper threshold for white color
lower_white = np.array([200])  # Adjust the threshold as needed
upper_white = np.array([255])
# Create a binary mask for white color
white_mask = cv2.inRange(gray, lower_white, upper_white)

# Find contours of white objects
contours, _ = cv2.findContours(white_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw the contours on the original image
blank = np.zeros_like(image)
threshold_area = 3000

# Initialize variables to store extreme points
rightmost = (w/2, h/2)
leftmost = (w/2, h/2)
topmost = (w/2, h/2)
bottommost = (w/2, h/2)

# Iterate over the contours and print only the big contours
for contour in contours:
    # Calculate the contour area
    area = cv2.contourArea(contour)
    
    # Check if the contour area is above the threshold
    if area > threshold_area:
        cv2.drawContours(blank, [contour], -1, (0, 255, 0), 2)
        
        # Find the leftmost, rightmost, topmost, and bottommost points of the contour
        leftmost_point = tuple(contour[contour[:, :, 0].argmin()][0])
        rightmost_point = tuple(contour[contour[:, :, 0].argmax()][0])
        topmost_point = tuple(contour[contour[:, :, 1].argmin()][0])
        bottommost_point = tuple(contour[contour[:, :, 1].argmax()][0])
        
        # Update extreme points
        if leftmost_point[0] < leftmost[0]:
            leftmost = leftmost_point
        if rightmost_point[0] > rightmost[0]:
            rightmost = rightmost_point
        if topmost_point[1] < topmost[1]:
            topmost = topmost_point
        if bottommost_point[1] > bottommost[1]:
            bottommost = bottommost_point

# Save the extreme points
extreme_points = [rightmost, topmost, leftmost, bottommost]
# Draw a polygon using the extreme points on top of the image
cv2.polylines(image, [np.array(extreme_points)], True, (0, 255, 0), 8)

array([[[ 26,  19,  29],
        [ 28,  21,  31],
        [ 29,  27,  39],
        ...,
        [  1,   4,   6],
        [  0,   3,   7],
        [  0,   3,   7]],

       [[ 25,  18,  28],
        [ 28,  21,  31],
        [ 30,  28,  40],
        ...,
        [  0,   3,   5],
        [  0,   2,   6],
        [  0,   2,   6]],

       [[ 23,  17,  29],
        [ 25,  19,  31],
        [ 28,  26,  38],
        ...,
        [  0,   5,   3],
        [  0,   2,   4],
        [  0,   0,   2]],

       ...,

       [[146, 133, 156],
        [148, 135, 158],
        [147, 139, 158],
        ...,
        [ 34,  39,  49],
        [ 28,  36,  45],
        [ 26,  34,  43]],

       [[150, 141, 163],
        [150, 141, 163],
        [154, 148, 162],
        ...,
        [ 37,  39,  50],
        [ 32,  36,  49],
        [ 28,  32,  45]],

       [[154, 145, 167],
        [155, 146, 168],
        [165, 159, 173],
        ...,
        [ 38,  40,  51],
        [ 36,  40,  53],
        [ 32,  36,  49]]

In [51]:
plt.imshow(blank)
plt.axis('off')
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [52]:
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2.imwrite('testimage.png', image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.axis('off')
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [53]:
ball_number = 2
human_number = 8
paste.paste_object(input_path, ball_number, human_number)
output_image = cv2.imread(output_path)
output_1 = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
plt.imshow(output_1)
plt.axis('off')
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [54]:
input_path = input_path.replace('src_images/0_aug', 'new_images')
paste.apply_random_augmentation(input_path)
output_image = cv2.imread(output_path)
output_2 = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)
plt.imshow(output_2)
plt.axis('off')
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [55]:
getcoco.convert2coco()

Converting into COCO ...


100%|██████████| 2/2 [00:00<00:00, 27.11it/s]

2 images
16 instances


In [56]:
annotation_path = '../../data/new_annotations/train_aug.json'
output_path_2 = '../../data/new_images/instaboost.png'
image = cv2.imread(input_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
with open(annotation_path, 'r') as f:
    annotation_data = json.load(f)
augmented_annotations, augmented_image = InstaBoost.get_new_data(annotation_data['annotations'], image)
augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
cv2.imwrite(output_path_2, augmented_image)
with open('instaboost_anno.json', 'w') as f:
    json.dump(augmented_annotations, f)
    f.close()

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)   
plt.imshow(image)
plt.axis('off')
plt.show

<function matplotlib.pyplot.show(*, block=None)>